In [1]:
from pynq import Overlay
from pynq import MMIO

In [2]:
ol = Overlay("../fobos.bit")

In [4]:
ol.ip_dict

{'axi_dma_0': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_0',
  'gpio': {},
  'interrupts': {'mm2s_introut': {'controller': 'axi_intc_0',
    'fullpath': 'axi_dma_0/mm2s_introut',
    'index': 0},
   's2mm_introut': {'controller': 'axi_intc_0',
    'fullpath': 'axi_dma_0/s2mm_introut',
    'index': 1}},
  'phys_addr': 1077936128,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'},
 'axi_dma_1': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_1',
  'gpio': {},
  'interrupts': {'s2mm_introut': {'controller': 'axi_intc_0',
    'fullpath': 'axi_dma_1/s2mm_introut',
    'index': 2}},
  'phys_addr': 1078001664,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'},
 'axi_iic_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_iic_0',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1096810496,
  'state': None,
  'type': 'xilinx.com:ip:axi_iic:2.0'},
 'axi_intc_0': {'addr_range': 65536,
  'driver': p

In [5]:
power_addr = ol.ip_dict["powermanager_0"]["phys_addr"]

In [6]:
power_addr_range = ol.ip_dict["powermanager_0"]["addr_range"]

In [7]:
power = MMIO(power_addr, power_addr_range)

In [8]:
# Registers
power_command       = 0x00
power_status        = 0x04
power_volt3v3       = 0x08
power_current3v3    = 0x0C
power_volt5v        = 0x10
power_current5v     = 0x14
power_avgvolt3v3    = 0x18
power_maxvolt3v3    = 0x1C
power_avgcurrent3v3 = 0x20
power_maxcurrent3v3 = 0x24
power_avgvolt5v     = 0x28
power_maxvolt5v     = 0x2C
power_avgcurrent5v  = 0x30
power_maxcurrent5v  = 0x34
power_volt_output   = 0x38
power_sampelcount   = 0x3C

# Commands
power_out_enable    = 0x01
power_clear         = 0x02
power_gain3v3cur    = 0x04
power_gain5vcur     = 0x10
power_trigenhw      = 0x40
power_trigsw        = 0x80

# Status
power_out_good      = 0x000
power_busy          = 0x002
#power_gain3v3cur    = 0x04
#power_gain5vcur     = 0x10
power_triggdhw      = 0x040
power_triggdsw      = 0x080
power_oflowsw       = 0x100


# Measurement Parameters
xadc_resolution = 65535  # 2^16 -1
xadc_max = 5 # Volt
xbp_shunt = 0.1 # Ohm, should be 0.1
xbp_gain = 25 # at least 25
pynq_vss = 5 # Volt Normally 5V
# Functions
def powerval(register):
    return pynq_vss * (( xadc_max/xadc_resolution * power.read(register) / (xbp_shunt * xbp_gain)))
def power_volt(register):
    return (xadc_max/xadc_resolution * power.read(register))
def power_current(register):
    return ( xadc_max/xadc_resolution * power.read(register) / (xbp_shunt * xbp_gain))





In [9]:
power.write(power_command, power_clear)

In [9]:
power.read(power_command)

0

In [46]:
power.write(power_command,0x00)

In [47]:
power.read(power_status)

0

In [26]:
power.read(power_volt3v3)

41212

In [27]:
power_volt(power_volt3v3)*1.055

3.315921263447013

In [16]:
power_volt(power_volt5v)*(1/power_volt(power_volt5v)/10+1.04)

0.08288352272727273

In [18]:
power.read(power_current3v3)

143

In [19]:
power.read(power_volt5v)

44852

In [18]:
power.read(power_current5v)

326

In [19]:
power.read(power_status)

0

In [20]:
power.write(power_command, power_out_enable)

In [18]:
power.write(power_volt_output, 0x00)

In [22]:
power.write(power_command, power_clear)

In [21]:
power.write(power_command, 0x0)

In [92]:
power.write(power_command, power_trigsw | power_out_enable)

In [100]:
power_volt(power_avgvolt3v3)

3.13751220703125

In [101]:
power_volt(power_maxvolt3v3)

3.14453125

In [108]:
power_volt(power_volt3v3)

3.1386566162109375

In [13]:
power_volt(power_volt5v)

0.011139085984588388

In [104]:
power_volt(power_avgvolt5v)

1.040496826171875

In [105]:
power_volt(power_maxvolt5v)

1.0465240478515625

In [99]:
power.write(power_command, power_out_enable)

In [48]:
power.read(power_status)

0

In [106]:
power.read(power_sampelcount)

1048575

In [ ]:
# JP1 must be in the VAR position
# J18 must be closed
# Nothing should be connected to the Target connector
import time
power.write(power_volt_output, 0x00)
power.write(power_command, power_out_enable)
print("Code, ADC Value, ADC Volts, Actual Volts")
for i in range(2**6):
    power.write(power_volt_output, i)
    time.sleep(1)
    t = 0
    while (power.read(power_status) & 0x01) != 0x01 and t <=10:
        t=t+1
        time.sleep(0.1)
    print(i, ",", power.read(power_volt5v), ",", power_volt(power_volt5v), "," , (365-i*5)/100 )    
power.write(power_command, 0x0)
time.sleep(1)
print("Off", ",", power.read(power_volt5v), ",", power_volt(power_volt5v), "," , 0)



Code, ADC Value, ADC Volts, Actual Volts
0 , 44861 , 3.4243533989471273 , 3.65
1 , 44182 , 3.37270160982681 , 3.6
2 , 43564 , 3.323414969100481 , 3.55
3 , 42872 , 3.273823147936217 , 3.5
4 , 42259 , 3.223086900129702 , 3.45
5 , 41602 , 3.1730373083085373 , 3.4
6 , 40941 , 3.1217669947356375 , 3.35
7 , 40238 , 3.075455863279164 , 3.3
8 , 39645 , 3.0241092545967803 , 3.25
9 , 39002 , 2.9790188448920425 , 3.2
10 , 38337 , 2.9253833829251543 , 3.15
11 , 37687 , 2.874799725337606 , 3.1
12 , 37020 , 2.826352330815595 , 3.05
13 , 36416 , 2.777065690089265 , 3.0
14 , 35711 , 2.7278553444724194 , 2.95
15 , 35080 , 2.6756694895857174 , 2.9
16 , 34530 , 2.6346990157930876 , 2.85
17 , 33877 , 2.583962767986572 , 2.8
18 , 33225 , 2.5357442587930112 , 2.75
19 , 32561 , 2.485847257190814 , 2.7
20 , 31949 , 2.4361028458075835 , 2.65
21 , 31290 , 2.388189517051957 , 2.6
22 , 30627 , 2.3356984817273214 , 2.55
23 , 29951 , 2.28549629968719 , 2.5
24 , 29336 , 2.242923628595407 , 2.45
25 , 28663 , 2.186465